<a href="https://colab.research.google.com/github/taejoonlab/BME203.2019/blob/master/DynamicProgrammingProtein.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Protein alignment example from Durbin, et al. (1998)

In [6]:
# Download the scoring matrix 
!wget https://raw.githubusercontent.com/taejoonlab/BME203.2019/master/data/BLOSUM62.txt
!ls

--2019-03-07 03:03:47--  https://raw.githubusercontent.com/taejoonlab/BME203.2019/master/data/BLOSUM62.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2122 (2.1K) [text/plain]
Saving to: ‘BLOSUM62.txt’

BLOSUM62.txt        100%[===================>]   2.07K  --.-KB/s    in 0s      

2019-03-07 03:03:48 (52.0 MB/s) - ‘BLOSUM62.txt’ saved [2122/2122]

BLOSUM62.txt  sample_data


In [0]:
# Check the matrix
!head -n 20 BLOSUM62.txt
!tail -n 5 BLOSUM62.txt

In [7]:
# Prepare the scoring matrix 'scores'

scores = dict()

f_mat = open('BLOSUM62.txt', 'r')
aa_tokens = []
for line in f_mat:
  if line.startswith('#'):
    continue
  elif line.startswith(' '):
    aa_tokens = line.strip().split()
  else:
    tmp_tokens = line.strip().split()
    for i in range(1, len(tmp_tokens)):
      scores['%s%s' % (aa_tokens[i-1], tmp_tokens[0])] = int(tmp_tokens[i])
f_mat.close()

print(scores['AA'], scores['A*'], scores['WW'])

4 -4 11


In [12]:
# Prepare the input sequences
seq1 = 'HEAGAWGHEE'
seq2 = 'PAWHEAE'

# Prepare the matrix for (1) score and (2) directionality
score_matrix = []
direction_matrix = []

for i in range(0, len(seq2)+1):
  tmp_score_list = []
  tmp_direction_list = []
  for j in range(0, len(seq1)+1):
    tmp_score_list.append(0)
    tmp_direction_list.append('N')
  score_matrix.append(tmp_score_list)
  direction_matrix.append(tmp_direction_list)

# Beautify the matrix output
def print_str_matrix(tmp_mat):
  for tmp_line in tmp_mat:
    print(" ".join([x.ljust(3, ' ') for x in tmp_line]))

def print_matrix(tmp_mat):
  for tmp_line in tmp_mat:
    print(tmp_line)

    
print_matrix(score_matrix)
print_str_matrix(direction_matrix)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
N   N   N   N   N   N   N   N   N   N   N  
N   N   N   N   N   N   N   N   N   N   N  
N   N   N   N   N   N   N   N   N   N   N  
N   N   N   N   N   N   N   N   N   N   N  
N   N   N   N   N   N   N   N   N   N   N  
N   N   N   N   N   N   N   N   N   N   N  
N   N   N   N   N   N   N   N   N   N   N  
N   N   N   N   N   N   N   N   N   N   N  


In [13]:
# Fill the matrix

for j in range(0, len(seq2)+1):
  for i in range(0, len(seq1)+1):
    if i == 0 and j == 0:
      score_matrix[i][j] = 0
    elif i == 0:
      # First column
      tmp_aa = '%s%s' % ('*', seq2[j-1])
      score_matrix[j][i] = score_matrix[j-1][i] + scores[tmp_aa]
      direction_matrix[j][i] = 'T'
    elif j == 0:
      # First row
      tmp_aa = '%s%s' % (seq1[i-1], '*')
      score_matrix[j][i] = score_matrix[j][i-1] + scores[tmp_aa]
      direction_matrix[j][i] = 'L'
    else:
      # For score
      tmp_aa_L = '%s%s' % (seq1[i-1], '*')
      tmp_score_L = score_matrix[j][i-1] + scores[tmp_aa_L]
      tmp_aa_T = '%s%s' % ('*', seq2[j-1])
      tmp_score_T = score_matrix[j-1][i] + scores[tmp_aa_T]
      tmp_aa_D = '%s%s' % (seq1[i-1], seq2[j-1])
      tmp_score_D = score_matrix[j-1][i-1] + scores[tmp_aa_D]
      
      max_score = max([tmp_score_L, tmp_score_T, tmp_score_D])
      score_matrix[j][i] = max_score
      
      # For direction
      tmp_direction = ''
      if max_score == tmp_score_L:
        tmp_direction += 'L'
      elif max_score == tmp_score_T:
        tmp_direction += 'T'
      if max_score == tmp_score_D:
        tmp_direction += 'D'
      direction_matrix[j][i] = tmp_direction

print("Score Matrix")
print_matrix(score_matrix)

print("Direction Matrix")
print_str_matrix(direction_matrix)

Score Matrix
[0, -4, -8, -12, -16, -20, -24, -28, -32, -36, -40]
[-4, -2, -5, -9, -13, -17, -21, -25, -29, -33, -37]
[-8, -6, -3, -1, -5, -9, -13, -17, -21, -25, -29]
[-12, -10, -7, -5, -3, -7, 2, -2, -6, -10, -14]
[-16, -4, -8, -9, -7, -5, -2, 0, 6, 2, -2]
[-20, -8, 1, -3, -7, -8, -6, -4, 2, 11, 7]
[-24, -12, -3, 5, 1, -3, -7, -6, -2, 7, 10]
[-28, -16, -7, 1, 3, 0, -4, -8, -6, 3, 12]
Direction Matrix
N   L   L   L   L   L   L   L   L   L   L  
T   D   D   LD  L   LD  L   L   L   LD  LD 
T   TD  D   D   L   LD  L   L   L   L   L  
T   TD  T   T   D   L   D   L   L   L   L  
T   D   L   TD  TD  D   T   D   D   L   L  
T   T   D   L   L   D   T   TD  T   D   LD 
T   T   T   D   L   LD  L   D   T   T   D  
T   T   TD  T   D   D   L   L   TD  TD  D  
